### First part of the hospital data preprocessing

In [ ]:
#%matplotlib widget
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

1) Functions

In [ ]:
def open_file(path):
    f=[]
    for k in sorted(listdir(path)):
        f.append(path + "/" + k) #add each file path in a folder in a list  
    return f

def df_function(path):
    df = pd.read_csv(path) #creation of the dataframe 
    df=df.sort_values(by=['horodate'], ascending=True) #the dates are sorted in ascending order
    df['horodate'] = pd.to_datetime(df['horodate'], format=format) #in the csv file the dates are str, transformation to datetime
    return df

def signal(path,path_ecg,format):
    #test=np.linspace(1,1,1)
    file_ppg=open_file(path) #open the pleth files
    file_ecg=open_file(path_ecg) #open the ecg files
    for i in range(len(file_ppg)):
        df_ppg=df_function(file_ppg[i]) #creation of df
        df_ecg=df_function(file_ecg[i])
        start_time_ecg = df_ecg['horodate'].iat[0] #start time is the first value in the 'horodate' column (bc ascending order)
        start_time_ppg = df_ppg['horodate'].iat[0]
        max_value=max(start_time_ecg,start_time_ppg)+timedelta(hours=1) #max value btw start times + 1 hour (before the patient is still settling in  
        df_closest_ppg = df_ppg.iloc[(df_ppg['horodate']-max_value).abs().argsort()[:1]] #closest date to max time for pleth
        df_closest_ecg = df_ecg.iloc[(df_ecg['horodate']-max_value).abs().argsort()[:1]] #same for ecg
        pleth=df_closest_ppg['pleth'].values[0].split(" ") #take the values of the signal (string) and split
        ecg=df_closest_ecg['ecg'].values[0].split(" ")
        if len(pleth)==641: #sometimes error=last value is null so remove 
            pleth=pleth[:-1]
        if len(ecg)==2561:
            ecg=ecg[:-1]
        pleth=list(map(int, pleth)) #put the strings in a list 
        ecg=list(map(float, ecg))
    return ecg, pleth

2) Return the signals

In [ ]:
import os
from os import listdir

#path='/home/claramacabiau/sadcsip_partage/Clara Macabiau/Pleth/Pleth_2018_2019/Pleth_id_6435.csv' #test with one file
#path_ecg='/home/claramacabiau/sadcsip_partage/Clara Macabiau/ECG/ECG_2018_2019/ECG_id_6435.csv'
path='/home/claramacabiau/sadcsip_partage/Clara Macabiau/Pleth/Pleth_2018_2019'
path_ecg='/home/claramacabiau/sadcsip_partage/Clara Macabiau/ECG/ECG_2018_2019'

#ecg, pleth=signal(path,path_ecg,format)

In [ ]:
## For one file only not the whole folder

# df_ppg=df_function(path)
# df_ecg=df_function(path_ecg)
# start_time_ecg = df_ecg['horodate'].iat[0]
# start_time_ppg = df_ppg['horodate'].iat[0]
# max_value=max(start_time_ecg,start_time_ppg)+timedelta(hours=1)
# df_closest_ppg = df_ppg.iloc[(df_ppg['horodate']-max_value).abs().argsort()[:1]]
# df_closest_ecg = df_ecg.iloc[(df_ecg['horodate']-max_value).abs().argsort()[:1]]
# pleth=df_closest_ppg['pleth'].values[0].split(" ")
# ecg=df_closest_ecg['ecg'].values[0].split(" ")
# if len(pleth)==641:
#     pleth=pleth[:-1]
# if len(ecg)==2561:
#     ecg=ecg[:-1]
# pleth=list(map(int, pleth))
# ecg=list(map(float, ecg))

3) Plot the signals

In [ ]:
plt.figure(1)
axe=np.linspace(1,len(pleth),len(pleth)) #sampling frequency 128 Hz
axe_e=np.linspace(1,len(ecg),len(ecg)) #sampling frequency 512 Hz
plt.subplot(2,1,1)
plt.title("PPG")
plt.plot(axe,pleth)
plt.subplot(2,1,2)
plt.title("ECG")
plt.plot(axe_e,ecg)
plt.show()

4) Creation of the dict for autoencoder

In [ ]:
dict_labels = dict(file_name = ['Pleth_id_5977','Pleth_id_6338','Pleth_id_6340','Pleth_id_6344','Pleth_id_6353','Pleth_id_6356',
'Pleth_id_6362','Pleth_id_6366','Pleth_id_6374','Pleth_id_6381','Pleth_id_6388','Pleth_id_6392','Pleth_id_6401',
'Pleth_id_6411','Pleth_id_6415','Pleth_id_6417','Pleth_id_6428','Pleth_id_6432','Pleth_id_6433','Pleth_id_6435'], 
labels = [1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1], 
date = ['2018-05-10 09:13:48','2018-08-25 23:54:17','2018-08-27 09:43:04','2018-08-28 04:09:04','2018-08-29 18:36:17',
'2018-08-29 23:05:51','2018-08-31 16:52:51','2018-09-02 00:52:51','2018-09-04 17:12:46','2018-09-06 18:43:16',
'2018-09-07 19:27:41','2018-09-08 16:21:18','2018-09-13 17:45:44','2018-09-15 08:07:58','2018-09-16 09:25:35',
'2018-09-16 21:54:07','2018-09-19 18:37:05','2018-09-20 23:43:36','2018-09-20 20:41:08','2018-09-20 21:19:37'], 
legend = ['1 = bad signal','0 = good signal'])